# Graph

> Obsidian App Graph Dashboard

In [ ]:
#| default_exp home.graph

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *
from fasthtml.svg import *
from memexplatform_obsidian.datastore import FileStore, NoteStore, DBStore, iter_files
from memexplatform_obsidian.commons import config, ExtensionTypes
import pandas as pd
import networkx as nx
from pyvis.network import Network
import re
from collections import Counter
from io import StringIO

In [ ]:

# node = nodes[0]

In [ ]:

#| export
def fetch_nodes_and_links(store, folders, extensions):
    """Fetch nodes and internal links from the database."""
    flds = ",".join(f"'{e}'" for e in folders)
    exts = ",".join(f"'{e.lstrip('.')}'" for e in extensions)

    db_nodes = list(store.db.query(f"""
        SELECT title, url, lockey FROM node
        WHERE is_folder=0
          AND parent IN ({flds})
          AND ext IN ({exts})
    """))

    db_links = list(store.db.query(f"""
        SELECT l.lockey, l.linked_lockey
        FROM links l
        JOIN node n1 ON l.lockey = n1.lockey
        JOIN node n2 ON l.linked_lockey = n2.lockey
        WHERE l.is_internal = 1
          AND n1.is_folder = 0
          AND n2.is_folder = 0
          AND n1.parent IN ({flds})
          AND n2.parent IN ({flds})
          AND n1.ext IN ({exts})
          AND n2.ext IN ({exts})
    """))

    return db_nodes, db_links

In [ ]:
#| export
def compute_degrees(db_links):
    """Compute directed in/out degrees and total degree."""
    out_degree = Counter()
    in_degree = Counter()

    for row in db_links:
        src = row.get('lockey')
        dst = row.get('linked_lockey')
        if not src or not dst:
            continue
        src, dst = str(src), str(dst)
        out_degree[src] += 1
        in_degree[dst] += 1

    total_degree = Counter()
    all_nodes = set(out_degree.keys()) | set(in_degree.keys())
    for node in all_nodes:
        total_degree[node] = out_degree[node] + in_degree[node]

    return in_degree, out_degree, total_degree

In [ ]:
#| export
def size_from_degree(deg, max_deg, min_size=10, max_size=50):
    """Scale node size linearly based on degree."""
    if max_deg <= 0:
        return min_size
    scaled = min_size + (deg / max_deg) * (max_size - min_size)
    return int(max(min_size, min(int(scaled), max_size)))

In [ ]:
#| export
def color_from_ratio(node_id, in_degree, out_degree):
    """Return color based on in/out ratio (red=in, blue=out)."""
    indeg = in_degree.get(node_id, 0)
    outdeg = out_degree.get(node_id, 0)
    total = indeg + outdeg
    if total == 0:
        return "#888888"  # grey for isolates

    ratio = indeg / total
    r = int(255 * ratio)
    g = 50
    b = int(255 * (1 - ratio))
    return f"rgb({r},{g},{b})"

In [ ]:
#| export
def build_network(db_nodes, db_links, in_degree, out_degree, total_degree):
    """Construct the pyvis Network object."""
    net = Network(
        height="1000px",
        width="100%",
        bgcolor="#1e1e1e",
        font_color="white",
        directed=True,
        notebook=False
    )

    net.barnes_hut(
        gravity=-20000,
        central_gravity=0.3,
        spring_length=120,
        spring_strength=0.05,
        damping=0.9
    )

    max_deg = max(total_degree.values()) if total_degree else 0

    # Add nodes
    for node in db_nodes:
        node_id = str(node.get('lockey'))
        if not node_id:
            continue
        deg = total_degree.get(node_id, 0)
        size = size_from_degree(deg, max_deg)
        color = color_from_ratio(node_id, in_degree, out_degree)

        net.add_node(
            node_id,
            label=node.get('title', ''),
            title=f"{node.get('title','')}\nIn: {in_degree.get(node_id,0)} Out: {out_degree.get(node_id,0)}",
            href=node.get('url', ''),
            url=node.get('url', ''),
            shape="dot",
            size=size,
            color={"background": color, "border": "#2f7acc"}
        )

    # Add edges
    for row in db_links:
        src = row.get('lockey')
        dst = row.get('linked_lockey')
        if src and dst:
            net.add_edge(str(src), str(dst), arrows="to")

    return net

In [ ]:
#| export
def inject_click_handlers(html_str):
    """Inject click & hover handlers into pyvis HTML string."""
    pattern = r'(network = new vis\.Network\(container, data, options\);)'

    click_code = r'''\1

                  // Add click event handlers
                  network.on("click", function (params) {
                      if (params.nodes.length > 0) {
                          var nodeId = params.nodes[0];
                          var clickedNode = nodes.get(nodeId);
                          
                          if (clickedNode.url || clickedNode.href) {
                              var targetUrl = clickedNode.url || clickedNode.href;
                              window.open(targetUrl);
                          }
                      }
                  });
                  
                  network.on("hoverNode", function (params) {
                      document.getElementById('mynetwork').style.cursor = 'pointer';
                  });
                  
                  network.on("blurNode", function (params) {
                      document.getElementById('mynetwork').style.cursor = 'default';
                  });'''

    html_str = re.sub(pattern, click_code, html_str)
    return html_str

In [ ]:
#| export
def export_network_to_buffer(net):
    """Export network HTML to a string (for NotStr)."""
    html_content = net.generate_html()  # <- returns HTML as string
    html_content = inject_click_handlers(html_content)
    return html_content

In [ ]:
#| export
def get_graph():
    fldrs = ('pages', 'journals', 'Clippings')
    extensions = ExtensionTypes.TEXTLIKE_EXTS
    store = DBStore(config) if config.ENABLE_DB_MODE else FileStore(config)

    db_nodes, db_links = fetch_nodes_and_links(store, fldrs, extensions)
    in_deg, out_deg, total_deg = compute_degrees(db_links)
    net = build_network(db_nodes, db_links, in_deg, out_deg, total_deg)
    graph_html_str = export_network_to_buffer(net)  # <- ready for ft.NotStr(graph_html_str)
    # graph_html_str
    return NotStr(graph_html_str)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()